# Web Scrapping

## Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from time import sleep

## Scraping ATP Top 100 Player Profiles

In [2]:
def parse_soup(sp):
    label_temp = []
    value = []

    # Players name and appending to value
    first_name = sp.find('div', class_='first-name').text
    last_name = sp.find('div', class_='last-name').text
    player_name = first_name + ' ' + last_name
    value.append(player_name)

    # This is pulling the Rank label and if it is for singles or doubles
    data_label_rank = sp.find('div', class_='data-label-text').text.strip()
    data_label_rank = re.sub('\r\n', ' ', data_label_rank)
    label_temp.append((re.sub(' +', ' ', data_label_rank)))

    # This is pull the value for the rank label
    data_number = sp.find('div', class_='data-number').text.strip()
    value.append(data_number)
    # rank = data_number  # Variable for class

    # This is pulling the labels Age, Turned Pro, Weight, and Height.
    for table_big_label in sp.find_all('div', class_='table-big-label'):
        label_temp.append(table_big_label.text)

    # This is pulling the values for the labels above

    for table_big_value in sp.find_all('div', class_='table-big-value'):
        x = table_big_value.text.strip()
        bad = "\n\r\n"
        for r in bad:
            x = x.replace(r, "")
        value.append(re.sub(' +', ' ', x))

    # This is pulling labels for Birthplace, Plays, Coach
    for table_label in sp.find_all('div', class_='table-label'):
        label_temp.append(table_label.text.strip())

    # This is pulling labels for Birthplace, Plays, Coach
    for table_value in sp.find_all('div', class_='table-value'):
        value.append(table_value.text.strip())

    return value

def player(value):

    # This loop is taking the string of variables and assigning them to their respected values
    label = ['fullname', 'rank', 'age', 'turned_pro', 'weight', 'height', 'birthplace', 'plays', 'coach']
    n = 0
    for name in label:
        globals()[name] = value[n]
        n += 1
    return fullname, rank, age, turned_pro, weight, height, birthplace, plays, coach

In [3]:
value_list = []  # Empty list to insert each player into in
html_text_rank = requests.get('https://www.atptour.com/en/rankings/singles').text
soup_rank = BeautifulSoup(html_text_rank, 'lxml')

url_link = []
html_1 = 'https://www.atptour.com'  # Needed this to add the link a pulled in from singles rank page

# This for loop is grabbing the links to the players overview if the link doesn't contain overview it passes
for a in soup_rank.find_all('a', href=True):
    link = a['href']
    if 'overview' in link:
        url = html_1 + link
        url_link.append(url)
    else:
        pass
print('all links grabbed')
# this for loop runs the functions above to grab the info from overview for each player and appends to value_list
for i in url_link:
    html_text = requests.get(i).text
    soup = BeautifulSoup(html_text, 'lxml')
    player(parse_soup(soup))
    value_list.append((fullname, int(rank), age, turned_pro, weight, height, birthplace, plays, coach))
    print(fullname, 'added to list')
# This is creating the dataframe
df = pd.DataFrame(value_list, columns=['fullname', 'rank', 'age', 'turned_pro', 'weight', 'height', 'birthplace', 'plays', 'coach'])
df.head()    

all links grabbed
Carlos Alcaraz added to list
Rafael Nadal added to list
Casper Ruud added to list
Stefanos Tsitsipas added to list
Novak Djokovic added to list
Andrey Rublev added to list
Felix Auger-Aliassime added to list
Daniil Medvedev added to list
Taylor Fritz added to list
Holger Rune added to list
Hubert Hurkacz added to list
Cameron Norrie added to list
Alexander Zverev added to list
Matteo Berrettini added to list
Pablo Carreno Busta added to list
Jannik Sinner added to list
Frances Tiafoe added to list
Marin Cilic added to list
Lorenzo Musetti added to list
Karen Khachanov added to list
Nick Kyrgios added to list
Denis Shapovalov added to list
Borna Coric added to list
Alex de Minaur added to list
Roberto Bautista Agut added to list
Diego Schwartzman added to list
Miomir Kecmanovic added to list
Grigor Dimitrov added to list
Francisco Cerundolo added to list
Daniel Evans added to list
Sebastian Korda added to list
Alejandro Davidovich Fokina added to list
Yoshihito Nishiok

,fullname,rank,age,turned_pro,weight,height,birthplace,plays,coach
0,Carlos Alcaraz,1,19 (2003.05.05),2018,163lbs(74kg),"6'0""(183cm)","El Palmar, Murcia, Spain","Right-Handed, Two-Handed Backhand",Juan Carlos Ferrero
1,Rafael Nadal,2,36 (1986.06.03),2001,187lbs(85kg),"6'1""(185cm)","Manacor, Mallorca, Spain","Left-Handed, Two-Handed Backhand","Carlos Moya, Marc Lopez, Gustavo Marcaccio"
2,Casper Ruud,3,24 (1998.12.22),2015,170lbs(77kg),"6'0""(183cm)","Oslo, Norway","Right-Handed, Two-Handed Backhand",Christian Ruud
3,Stefanos Tsitsipas,4,24 (1998.08.12),2016,198lbs(90kg),"6'4""(193cm)","Athens, Greece","Right-Handed, One-Handed Backhand","Apostolos Tsitsipas, Mark Philippoussis"
4,Novak Djokovic,5,35 (1987.05.22),2003,170lbs(77kg),"6'2""(188cm)","Belgrade, Serbia","Right-Handed, Two-Handed Backhand",Goran Ivanisevic


## Scraping Livescore.in for point by point data

In [9]:
%%time
# This function works on livescore.in and grabs the links to all of the tournaments in the archive tab.
# No need to run. Works but is not clean, the next cell is the better on to run.
main = []
tennis = "/tennis/atp-singles/"

url = "https://www.livescore.in/tennis/atp-singles/"
url_base = "https://www.livescore.in"
# get html content
r = requests.get(url)
html_content_main = r.text
# print(html_content)
# parse the html using BeautifulSoup
soup = BeautifulSoup(html_content_main, "html.parser")
# we are looking for all atp-singles matches first
for i in soup.find_all("a", href=True):
    # this holder list will eventually be appended to the main list
    # going to be holding all of the main links to match data
    holder = []
    extension = i['href']
    if tennis in extension:
        new_url = url_base + extension
        # sending our first main link to the holder list
        holder.append(new_url)
        r_level_1 = requests.get(new_url)
        html_content_level_1 = r_level_1.text
        soup = BeautifulSoup(html_content_level_1, "html.parser")
        # going into each of the tournaments and looking for the archive tab
        for i in soup.find_all("a", href=True):
            extension_2 = i['href']
            # looking to see which links contain 'archive'
            if 'archive' in extension_2:
                arch_url = url_base + extension_2
                # grabbing those links html and looking for past tournament years
                r_level_2 = requests.get(arch_url)
                html_content_level_2 = r_level_2.text
                soup = BeautifulSoup(html_content_level_2, "html.parser")
                # we are we are going through the links in the archive tab
                # and only collecting past years in the 2000s which all have a -20
                for i in soup.find_all("a", href=True):
                    match_year_url = i['href']
                    if '-20' in match_year_url:
                        holder.append(match_year_url)
                    else:
                        pass
            else:
                pass
    else:
        pass
    main.append(holder)


CPU times: user 18.8 s, sys: 374 ms, total: 19.2 s
Wall time: 3min 8s


In [13]:
# This is an optimized and better output than the cell above... keeping both for reference for now

main = []
tennis = "/tennis/atp-singles/"

url = "https://www.livescore.in/tennis/atp-singles/"
url_base = "https://www.livescore.in"
# get html content
r = requests.get(url)
html_content_main = r.text
# parse the html using BeautifulSoup
soup = BeautifulSoup(html_content_main, "html.parser")
# year = It looks values in the string from 2012 through 2023. 
# Not all tournaments have point by point data so will need to filter later.
year = re.compile(r'(?:20(12|13|14|15|16|17|18|19|20|21|22|23))')
# we are looking for all atp-singles matches first
holder = [url_base + i['href'] for i in soup.find_all("a", href=True) if tennis in i['href']]
# level 1
for new_url in holder:
    main.append(new_url + 'results/')
    r_level_1 = requests.get(new_url)
    html_content_level_1 = r_level_1.text
    soup = BeautifulSoup(html_content_level_1, "html.parser")
    # level 2
    if len([url_base + i['href'] for i in soup.find_all("a", href=True) if 'archive' in i['href']])>0:
        arch_url = [url_base + i['href'] for i in soup.find_all("a", href=True) if 'archive' in i['href']][0]
        r_level_2 = requests.get(arch_url)
        html_content_level_2 = r_level_2.text
        soup = BeautifulSoup(html_content_level_2, "html.parser")
        # level 3
        match_year_url = [i['href'] for i in soup.find_all("a", href=True) if year.search(i['href'])]
        for i in match_year_url:
            final_url = url_base + i + 'results/'
            main.append(final_url)
    print(f'{new_url} has been stored')

https://www.livescore.in/tennis/atp-singles/ has been stored
https://www.livescore.in/tennis/atp-singles/ has been stored
https://www.livescore.in/tennis/atp-singles/adelaide/ has been stored
https://www.livescore.in/tennis/atp-singles/amersfoort/ has been stored
https://www.livescore.in/tennis/atp-singles/antalya/ has been stored
https://www.livescore.in/tennis/atp-singles/antwerp/ has been stored
https://www.livescore.in/tennis/atp-singles/atp-cup/ has been stored
https://www.livescore.in/tennis/atp-singles/auckland/ has been stored
https://www.livescore.in/tennis/atp-singles/australian-open/ has been stored
https://www.livescore.in/tennis/atp-singles/bangkok/ has been stored
https://www.livescore.in/tennis/atp-singles/barcelona/ has been stored
https://www.livescore.in/tennis/atp-singles/basel/ has been stored
https://www.livescore.in/tennis/atp-singles/bastad/ has been stored
https://www.livescore.in/tennis/atp-singles/beijing/ has been stored
https://www.livescore.in/tennis/atp-si

In [15]:
with open("output.txt", "w") as f:
    f.writelines(str(i) + '\n' for i in main)

In [16]:
url = "https://www.livescore.in/tennis/atp-singles/auckland-2019/results/"
_temp = []
# get html content
r = requests.get(url)
html_content = r.text
print(html_content)
# soup = BeautifulSoup(html_content, "html.parser")

# # find class="lmc__template"
# for i in soup.find_all("a", href=True):

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">
<head>
<meta http-equiv="content-type" content="text/html; charset=utf-8" />
<meta http-equiv="X-UA-Compatible" content="IE=11" />
<title>ATP Auckland 2019 - Tennis, ATP - Singles | Livescore.in</title>
<link rel="preload" href="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" as="script">
<link rel="preload" href="https://cdn.cookielaw.org/vendorlist/iab2Data.json" as="fetch" crossorigin>
<link rel="preload" href="/res/image/data/dlvrQ5hD-fyjVgadA.png" as="image">
<noscript>
<meta http-equiv="refresh" content="0;URL='https://www.flashscore.mobi/'" />
</noscript>
<meta name="keywords" content="" />
<meta name="description" content="" />
<meta name="copyright" content="Copyright (c) 2008-2023 Livesport s.r.o." />
<meta name="robots" content="index,follow" />
<meta name="viewport" content="width=device-width, initial-

In [31]:
import json

url = "https://www.livescore.in/match/bZKdaaKU/#/match-summary/point-by-point/0"
_temp = []
# get html content
r = requests.get(url).text
soup = BeautifulSoup(url, 'lxml')
for test in soup.find_all('div', class_='matchHistoryRow__score '):
    _temp.append(test.text)

print(_temp)

[]


/Users/Stephen1/opt/anaconda3/envs/scrape/lib/python3.10/site-packages/bs4/__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [34]:
url = "https://www.livescore.in/match/bZKdaaKU/#/match-summary/point-by-point/0"

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    match_summary = soup.find("div", {"class": "matchHistoryRow__score "})
    print(match_summary)
except r.exceptions.RequestException as e:
    print("Error", e)


None


In [45]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)

In [47]:
# Navigate to the website
browser.visit("https://www.livescore.in/match/bZKdaaKU/#/match-summary/point-by-point/0")
browser.is_element_present_by_css('div.list_text', wait_time=3)
html = browser.html
news_soup = soup(html, 'html.parser')
try:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    match_summary = soup.find("div", {"class": "matchHistoryRow__score "})
    print(match_summary)
except r.exceptions.RequestException as e:
    print("Error", e)
# Close the webdriver
# browser.quit()


print(match_summary)

None
None


In [54]:
from selenium import webdriver

url = 'https://www.livescore.in/match/bZKdaaKU/#/match-summary/point-by-point/0'
driver = webdriver.Chrome()
driver.get(url)
page_source = driver.page_source

soup = soup(page_source, 'html.parser')
try:
    match_summary = soup.find("div", {"class": "matchHistoryRow__score "})
    print(match_summary)
except r.exceptions.RequestException as e:
    print("Error", e)
# Close the webdriver
browser.quit()



AttributeError: 'str' object has no attribute 'exceptions'

In [89]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Set options for Google Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument('--disable-extensions')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Start a webdriver instance
driver = webdriver.Chrome(options=options)

# Navigate to the website
driver.get("https://www.livescore.in/match/bZKdaaKU/#/match-summary/point-by-point/0")

# Wait for the page to load
driver.implicitly_wait(5)

set_info = []
set_count = driver.find_elements(By.CLASS_NAME, "subTabs__tab")
for i in set_count:
    current_set = i.text
    link = i.get_attribute('href')
    set_items = {
        'set': current_set,
        'link': link
    }
    set_info.append(set_items)



game = []
game_data = driver.find_elements(By.CLASS_NAME, "matchHistoryRow")
for game_data in game_data:
    player_1 = game_data.find_element(By.CLASS_NAME, 'matchHistoryRow__servis matchHistoryRow__highlight matchHistoryRow__home')
    print(player_1)
    # if player_1 == None:
    #     serving = 'P2'
    # else: 
    #     serving = 'P1'
    server = {
        'Server': player_1
    }
    game.append(server)
# //*[@id="detail"]/div[8]
# //*[@id="detail"]/div[9]/div[2]/div[2]
# //*[@id="detail"]/div[9]/div[6]/div[2]/div
# //*[@id="detail"]/div[9]/div[2]/div[2]/div
# //*[@id="detail"]/div[9]/div[4]/div[4]/div
# //*[@id="detail"]/div[9]/div[8]/div[4]/div
# Close the webdriver
driver.quit()
df = pd.DataFrame(game)
display(df)



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".matchHistoryRow__servis matchHistoryRow__highlight matchHistoryRow__home"}
  (Session info: headless chrome=109.0.5414.119)
Stacktrace:
0   chromedriver                        0x0000000104417fa8 chromedriver + 4886440
1   chromedriver                        0x0000000104395643 chromedriver + 4351555
2   chromedriver                        0x0000000103fe3b27 chromedriver + 477991
3   chromedriver                        0x000000010402876c chromedriver + 759660
4   chromedriver                        0x00000001040289c1 chromedriver + 760257
5   chromedriver                        0x000000010401a506 chromedriver + 701702
6   chromedriver                        0x000000010404e5fd chromedriver + 914941
7   chromedriver                        0x000000010401a3dd chromedriver + 701405
8   chromedriver                        0x000000010404e78e chromedriver + 915342
9   chromedriver   